# Main Configs

In [1]:
small_df = False
batch_size = 1
width, height = 300, 300

# Setup and Imports

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir /usr/lib/python3.7/metrics
!cp -R /content/drive/MyDrive/BA/Notebooks/2_Experiment/review_object_detection_metrics-main/src /usr/lib/python3.7/metrics/src

In [4]:
!pip install PyQt5
!pip install -qU torch_snippets

     |████████████████████████████████| 8.3 MB 22.7 MB/s 
     |████████████████████████████████| 338 kB 65.4 MB/s 
     |████████████████████████████████| 59.9 MB 145.5 MB/s 
     |████████████████████████████████| 48 kB 5.6 MB/s 
     |████████████████████████████████| 948 kB 51.4 MB/s 
     |████████████████████████████████| 78 kB 8.8 MB/s 
     |████████████████████████████████| 60 kB 8.6 MB/s 
     |████████████████████████████████| 229 kB 86.0 MB/s 
     |████████████████████████████████| 58 kB 7.2 MB/s 
     |████████████████████████████████| 10.9 MB 83.8 MB/s 
     |████████████████████████████████| 51 kB 7.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.4.0 which is incompatible.


In [5]:
import copy
import glob
import torch
import time
import statistics
import cv2
import pandas as pd
import IPython

from os.path import join
from torch_snippets import *
from PIL import Image
from metrics.src.evaluators import coco_evaluator, pascal_voc_evaluator
from metrics.src.bounding_box import BoundingBox
from metrics.src.utils.enumerators import BBFormat, BBType, CoordinatesType, MethodAveragePrecision
from skimage import data
from skimage.color import rgb2hsv, rgb2luv, rgb2lab
from skimage.util import random_noise

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
IMAGE_ROOT = '/content/drive/MyDrive/BA/dataset/bus-trucks/images'
OUTPUT_REPORTS = '/content/drive/MyDrive/BA/Notebooks/2_Experiment/Output_Reports'
OUTPUT_MODELS = '/content/drive/MyDrive/BA/Notebooks/2_Experiment/Output_Models'
output_testing_report = 'testing_faster_rcnn_mobilenetv3_large_320.xlsx'
output_training_report = 'faster_rcnn_mobilenetv3_large_320.csv'
output_model_name = 'faster_rcnn_mobilenetv3_large_320.pt'

In [8]:
df_train = pd.read_csv('/content/drive/MyDrive/BA/dataset/Experimente/df_80_20_train_tf.csv')
df_test = pd.read_csv('/content/drive/MyDrive/BA/dataset/Experimente/df_80_20_test_tf.csv')
df_test.head()

,filename,class,xmin,xmax,ymin,ymax
0,00013f14dd4e168f.jpg,Bus,0.287500,0.999375,0.194184,0.999062
1,0002914fa805e227.jpg,Truck,0.061250,0.966875,0.125399,0.974495
2,0005f203463a13a8.jpg,Truck,0.000000,0.700000,0.187778,0.998889
3,00066517f9d814f9.jpg,Truck,0.000000,0.588867,0.069892,0.998208
4,000812dcf304a8e7.jpg,Bus,0.059375,0.848750,0.029936,0.958660


In [9]:
label2target = {l:t+1 for t,l in enumerate(df_train['class'].unique())}
label2target['background'] = 0
target2label = {t:l for l,t in label2target.items()}
label2target = {v: k for k, v in target2label.items()}
background_class = label2target['background']
num_classes = len(label2target)

print(target2label)

{1: 'Bus', 2: 'Truck', 0: 'background'}


In [10]:
df_report = pd.read_csv(join(OUTPUT_REPORTS, output_training_report))
df_report.head()

,epoch,time,loss,loss_classifier,loss_box_reg,loss_objectness,loss_rpn_box_reg
0,0,0.532456,1.826923,1.363694,0.373133,0.085319,0.004777
1,0,0.114823,1.570877,1.115933,0.405240,0.043967,0.005737
2,0,0.120514,0.967652,0.608759,0.279076,0.075146,0.004671
3,0,0.105697,0.982080,0.448771,0.496579,0.033435,0.003295
4,0,0.112777,0.755007,0.352983,0.363649,0.035939,0.002436


In [11]:
def preprocess_image(img):
  img = torch.tensor(img).permute(2,0,1)
  return img.to(device).float()

class OpenDataset(torch.utils.data.Dataset):
  w, h = width, height
  def __init__(self, df, image_dir=IMAGE_ROOT):
    self.image_dir = image_dir
    self.files = glob.glob(self.image_dir+'/*')
    self.df = df
    self.image_infos = df['filename'].unique()

  def __getitem__(self, ix):

    #filename	class	xmin	xmax	ymin	ymax

    # # load images and masks
    image_id = self.image_infos[ix]
    img_path = find(image_id, self.files)
    img = Image.open(img_path).convert('RGB')
    img = np.array(img.resize((self.w, self.h), resample=Image.BILINEAR))/255.

    data = self.df[self.df['filename'] == image_id]
    labels = data['class'].values.tolist()
    data = data[['xmin','ymin','xmax','ymax']].values
    data[:,[0,2]] *= self.w
    data[:,[1,3]] *= self.h
    boxes = data.astype(np.uint32).tolist() # convert to absolute coordinates
    # torch FRCNN expects ground truths as a dictionary of tensors
    target = {}
    target["boxes"] = torch.Tensor(boxes).float()
    target["labels"] = torch.Tensor([label2target[i] for i in labels]).long()
    img = preprocess_image(img)

    return img, target
  
  @classmethod
  def get_image_meta(self, ix):
    image_id = self.image_infos[ix]
    img_path = find(image_id, self.files)
    img = Image.open(img_path).convert('RGB')
    img = np.array(img)/255.

    return image_id, img.shape

  def collate_fn(self, batch):
    return tuple(zip(*batch)) 

  def __len__(self):
    return len(self.image_infos)

In [12]:
if small_df:
  df_test = df_test[:50]

In [13]:
test_ds = OpenDataset(df_test)
test_loader = DataLoader(test_ds, batch_size=batch_size, collate_fn=test_ds.collate_fn, drop_last=True)

In [14]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

PATH = join(OUTPUT_MODELS, output_model_name)

# Load
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=False)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.load_state_dict(torch.load(PATH, map_location=device))
model.to(device);

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth


  0%|          | 0.00/21.1M [00:00<?, ?B/s]

In [15]:
from torchvision.ops import nms
def decode_output(output):
  'convert tensors to numpy arrays'
  bbs = output['boxes'].cpu().detach().numpy().astype(np.uint16)
  labels = np.array([target2label[i] for i in output['labels'].cpu().detach().numpy()])
  confs = output['scores'].cpu().detach().numpy()
  ixs = nms(torch.tensor(bbs.astype(np.float32)), torch.tensor(confs), 0.05)
  bbs, confs, labels = [tensor[ixs] for tensor in [bbs, confs, labels]]

  if len(ixs) == 1:
      bbs, confs, labels = [np.array([tensor]) for tensor in [bbs, confs, labels]]
  return bbs.tolist(), confs.tolist(), labels.tolist()

# Size Analysis

In [16]:
bounding_boxes_detected = []
bounding_boxes_gt = []
model.eval()

for ix, (images, targets) in enumerate(test_loader):
  images = [torch.tensor(im).type(torch.FloatTensor).to(device) for im in images]
  outputs = model(images)
  outputs = decode_output(outputs[0])

  #Detected
  for i, output in enumerate(outputs[0]):
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = outputs[2][i],
      coordinates       = output,
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.DETECTED,
      confidence        = outputs[1][i],
      format            = BBFormat.XYX2Y2
    )
    bounding_boxes_detected.append(bb)


  #Ground Truth
  for i, box in enumerate(targets[0]['boxes']):
    cls = target2label[int(targets[0]['labels'][i])]
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    bounding_boxes_gt.append(bb)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [17]:
coco_evaluator.get_coco_summary(
    groundtruth_bbs = bounding_boxes_gt,
    detected_bbs = bounding_boxes_detected,
    small_size = 146,
    medium_size = 228
)

{'AP': 0.4290140895847271,
 'AP50': 0.6093659323250014,
 'AP75': 0.48446757313110056,
 'APlarge': 0.7152754014528375,
 'APmedium': 0.6618762098248865,
 'APsmall': 0.23745070073719793,
 'AR1': 0.43372733719639084,
 'AR10': 0.4806976208916135,
 'AR100': 0.4806976208916135,
 'ARlarge': 0.771267931162964,
 'ARmedium': 0.7208207642031171,
 'ARsmall': 0.28920182032803177}

# Full COCO

In [18]:
df = coco_evaluator.get_coco_metrics(
        bounding_boxes_gt,
        bounding_boxes_detected,
        iou_threshold=0.5,
        area_range=(0, np.inf),
        max_dets=100,
)
df = pd.DataFrame.from_dict(df)
df.head()

,Bus,Truck
class,Bus,Truck
precision,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
recall,"[0.00043233895373973193, 0.0008646779074794639...","[0.0003996802557953637, 0.0007993605115907274,..."
AP,0.636113,0.582619
interpolated precision,"[1.0, 1.0, 1.0, 1.0, 0.9825581395348837, 0.982...","[1.0, 1.0, 1.0, 1.0, 0.9935897435897436, 0.993..."


# PASCAL VOL

In [19]:
df = pascal_voc_evaluator.get_pascalvoc_metrics(
    bounding_boxes_gt,
    bounding_boxes_detected,
    iou_threshold= 0.1,
    method=MethodAveragePrecision.EVERY_POINT_INTERPOLATION
)

df = pd.DataFrame.from_dict(df)
df.head()

,per_class,mAP
Bus,"{'precision': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1...",0.673317
Truck,"{'precision': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1...",0.673317


# Color Analysis

In [20]:
def image_light(img, thrshld = 127):
  if type(img) is torch.Tensor:
    img = img.cpu().permute(1,2,0).numpy()
  rgb = img * 255
  rgb = np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])
  
  is_light = np.mean(rgb) > thrshld

  # Light: 0
  # Dark: 1

  # return 0 if is_light else 1
  return {'value': np.mean(rgb),
          'brightness': 'light' if is_light else 'dark'
          }

def avg_img_color(img):
  if type(img) is torch.Tensor:
    img = img.cpu().permute(1,2,0).numpy()
  rgb_avg = img.mean(axis=(0, 1)) 
  hsv_avg = rgb2hsv(img).mean(axis=(0, 1))
  luv_avg = rgb2luv(img).mean(axis=(0, 1))
  lab_avg = rgb2lab(img).mean(axis=(0, 1))

  return {
    'rgb_avg_r': float(rgb_avg[0]),
    'rgb_avg_g': float(rgb_avg[1]),
    'rgb_avg_b': float(rgb_avg[2]),
    'hsv_avg_h': float(hsv_avg[0]),
    'hsv_avg_s': float(hsv_avg[1]),
    'hsv_avg_v': float(hsv_avg[2])
  }

def get_sharpness(img):
  if type(img) is torch.Tensor:
    img = img.cpu().permute(1,2,0).numpy()
  i = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])

  gy, gx = np.gradient(i)
  gnorm = np.sqrt(gx**2 + gy**2)
  sharpness = np.average(gnorm)
  return sharpness

In [21]:
def bb_intersection_over_union(boxA, boxB):

  xA = max(boxA[0], boxB[0])
  yA = max(boxA[1], boxB[1])
  xB = min(boxA[2], boxB[2])
  yB = min(boxA[3], boxB[3])

  interArea = (xB - xA) * (yB - yA)

  boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
  boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

  iou = interArea / float(boxAArea + boxBArea - interArea)

  return iou


def detected(preds, gt, iou_threshold ):
  det = False
  max_conf = 0
  for i, pred in enumerate(preds):
    if pred[0] == gt[0]:
      if pred[1] == gt[1]:
        iuo = bb_intersection_over_union(pred[-4:], gt[-4:])
        if iuo >= iou_threshold:
          det = True
          if pred[2] > max_conf:
            max_conf = pred[2]

  return det, max_conf

In [22]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    string = f'\r{prefix} |{bar}| {percent}% {suffix}'
    out.update(IPython.display.Pretty(string))
    # Print New Line on Complete
    if iteration == total: 
        print()

In [23]:
%%time

blur_filter = 1/9 *  torch.tensor([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
metadata = []
n_items = len(test_loader)
out = display(IPython.display.Pretty('Starting'), display_id=True)

model.eval()

for ix, (images, targets) in enumerate(test_loader):
  # if ix == 10:
  #   break
  img = images[0]
  t = targets[0]['boxes']
  img_ = Image.fromarray(np.uint8(img.cpu().permute(1, 2, 0).numpy() * 255))


  # plain image
  c_img = image_light(img)
  c_avg_img = avg_img_color(img)

  preds = []
  outputs = model(images)
  for i_output, output in enumerate(outputs):
    bbs, confs, labels = decode_output(output)
    for i_bb, bb in enumerate(bbs):
      X1, Y1, X2, Y2 = bb
      preds.append([ix, labels[i_bb], confs[i_bb], X1, Y1, X2, Y2])


  for i, box in enumerate(targets[0]['boxes']):
    cls = target2label[int(targets[0]['labels'][i])]

    X1, Y1, X2, Y2 = box.numpy().astype(int)

    # object frame
    obj_frame = img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls, 1, X1, Y1, X2, Y2],
        0.6
    )
    metadata.append({'image_id': ix,
                    'image_version': 'standard',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls
                   }
                  )
###########################################################################################

  # blurred image
  blurred_img = cv2.filter2D(img.cpu().permute(1, 2, 0).numpy(), -1, kernel = blur_filter.numpy())
  c_img = image_light(blurred_img)
  c_avg_img = avg_img_color(blurred_img)

  blurred_imgs = [torch.from_numpy(blurred_img).permute(2, 0, 1)]
  blurred_imgs = [torch.tensor(im).type(torch.FloatTensor).to(device) for im in blurred_imgs]
  blurred_img = blurred_imgs[0]

  preds = []
  outputs = model(blurred_imgs) 

  for i_output, output in enumerate(outputs):
    bbs, confs, labels = decode_output(output)
    for i_bb, bb in enumerate(bbs):
      X1, Y1, X2, Y2 = bb
      preds.append([ix, labels[i_bb], confs[i_bb], X1, Y1, X2, Y2])


  for i, box in enumerate(targets[0]['boxes']):
    cls = target2label[int(targets[0]['labels'][i])]

    X1, Y1, X2, Y2 = box.numpy().astype(int)

    # object frame
    obj_frame = blurred_img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls, 1, X1, Y1, X2, Y2],
        0.6
    )
    metadata.append({'image_id': ix,
                    'image_version': 'blurred',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls
                   }
                  )
###########################################################################################
  # noisy image
  noise_img = random_noise(img.cpu().permute(1, 2, 0).numpy(), mode='s&p',amount=0.05)
  c_img = image_light(noise_img)
  c_avg_img = avg_img_color(noise_img)

  noisy_imgs = [torch.from_numpy(noise_img).permute(2, 0, 1)]
  noisy_imgs = [torch.tensor(im).type(torch.FloatTensor).to(device) for im in noisy_imgs]
  noise_img = noisy_imgs[0]

  preds = []
  outputs = model(noisy_imgs) 

  for i_output, output in enumerate(outputs):
    bbs, confs, labels = decode_output(output)
    for i_bb, bb in enumerate(bbs):
      X1, Y1, X2, Y2 = bb
      preds.append([ix, labels[i_bb], confs[i_bb], X1, Y1, X2, Y2])


  for i, box in enumerate(targets[0]['boxes']):
    cls = target2label[int(targets[0]['labels'][i])]

    X1, Y1, X2, Y2 = box.numpy().astype(int)

    # object frame
    obj_frame = noise_img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls, 1, X1, Y1, X2, Y2],
        0.6
    )
    metadata.append({'image_id': ix,
                    'image_version': 'noisy',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls
                   }
                  )
###########################################################################################
  # bright image

  new_image = PIL.ImageEnhance.Brightness(img_).enhance(2)
  bright_image = np.array(new_image) / 255
  bright_imgs = [torch.from_numpy(bright_image).permute(2, 0, 1)]
  bright_imgs = [torch.tensor(im).type(torch.FloatTensor).to(device) for im in bright_imgs]
  bright_img = bright_imgs[0]

  c_img = image_light(bright_image)
  c_avg_img = avg_img_color(bright_image)



  preds = []
  outputs = model(bright_imgs) 

  for i_output, output in enumerate(outputs):
    bbs, confs, labels = decode_output(output)
    for i_bb, bb in enumerate(bbs):
      X1, Y1, X2, Y2 = bb
      preds.append([ix, labels[i_bb], confs[i_bb], X1, Y1, X2, Y2])


  for i, box in enumerate(targets[0]['boxes']):
    cls = target2label[int(targets[0]['labels'][i])]

    X1, Y1, X2, Y2 = box.numpy().astype(int)

    # object frame
    obj_frame = bright_img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls, 1, X1, Y1, X2, Y2],
        0.6
    )
    metadata.append({'image_id': ix,
                    'image_version': 'bright',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls
                   }
                  )
    
###########################################################################################
  # contrast image

  new_image = PIL.ImageEnhance.Contrast(img_).enhance(2)
  contrast_image = np.array(new_image) / 255
  contrast_imgs = [torch.from_numpy(contrast_image).permute(2, 0, 1)]
  contrast_imgs = [torch.tensor(im).type(torch.FloatTensor).to(device) for im in contrast_imgs]
  contrast_img = contrast_imgs[0]

  c_img = image_light(contrast_image)
  c_avg_img = avg_img_color(contrast_image)



  preds = []
  outputs = model(contrast_imgs)

  for i_output, output in enumerate(outputs):
    bbs, confs, labels = decode_output(output)
    for i_bb, bb in enumerate(bbs):
      X1, Y1, X2, Y2 = bb
      preds.append([ix, labels[i_bb], confs[i_bb], X1, Y1, X2, Y2])


  for i, box in enumerate(targets[0]['boxes']):
    cls = target2label[int(targets[0]['labels'][i])]

    X1, Y1, X2, Y2 = box.numpy().astype(int)

    # object frame
    obj_frame = contrast_img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls, 1, X1, Y1, X2, Y2],
        0.6
    )
    metadata.append({'image_id': ix,
                    'image_version': 'contrast',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls
                   }
                  )
###########################################################################################
  # sharp image

  new_image = PIL.ImageEnhance.Sharpness(img_).enhance(3)
  sharp_image = np.array(new_image) / 255
  sharp_imgs = [torch.from_numpy(sharp_image).permute(2, 0, 1)]
  sharp_imgs = [torch.tensor(im).type(torch.FloatTensor).to(device) for im in sharp_imgs]
  sharp_img = sharp_imgs[0]

  c_img = image_light(sharp_image)
  c_avg_img = avg_img_color(sharp_image)



  preds = []
  outputs = model(sharp_imgs)

  for i_output, output in enumerate(outputs):
    bbs, confs, labels = decode_output(output)
    for i_bb, bb in enumerate(bbs):
      X1, Y1, X2, Y2 = bb
      preds.append([ix, labels[i_bb], confs[i_bb], X1, Y1, X2, Y2])


  for i, box in enumerate(targets[0]['boxes']):
    cls = target2label[int(targets[0]['labels'][i])]

    X1, Y1, X2, Y2 = box.numpy().astype(int)

    # object frame
    obj_frame = sharp_img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls, 1, X1, Y1, X2, Y2],
        0.6
    )
    metadata.append({'image_id': ix,
                    'image_version': 'sharp',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls
                   }
                  )
###########################################################################################
  # Dark image

  new_image = PIL.ImageEnhance.Brightness(img_).enhance(0.4)
  dark_image = np.array(new_image) / 255
  dark_imgs = [torch.from_numpy(dark_image).permute(2, 0, 1)]
  dark_imgs = [torch.tensor(im).type(torch.FloatTensor).to(device) for im in dark_imgs]
  dark_img = dark_imgs[0]

  c_img = image_light(dark_image)
  c_avg_img = avg_img_color(dark_image)



  preds = []
  outputs = model(dark_imgs)

  for i_output, output in enumerate(outputs):
    bbs, confs, labels = decode_output(output)
    for i_bb, bb in enumerate(bbs):
      X1, Y1, X2, Y2 = bb
      preds.append([ix, labels[i_bb], confs[i_bb], X1, Y1, X2, Y2])


  for i, box in enumerate(targets[0]['boxes']):
    cls = target2label[int(targets[0]['labels'][i])]

    X1, Y1, X2, Y2 = box.numpy().astype(int)

    # object frame
    obj_frame = dark_img[:3, Y1:Y2, X1:X2]

    # colors
    c_obj = image_light(obj_frame)
    c_avg_obj = avg_img_color(obj_frame)

    # Area
    bb = BoundingBox(
      image_name        = str(ix),
      class_id          = cls,
      coordinates       = list(box),
      type_coordinates  = CoordinatesType.ABSOLUTE,
      bb_type           = BBType.GROUND_TRUTH,
      confidence        = None,
      format            = BBFormat.XYX2Y2
    )
    
    # detected
    det, conf = detected(
        preds,
        [ix, cls, 1, X1, Y1, X2, Y2],
        0.6
    )
    metadata.append({'image_id': ix,
                    'image_version': 'dark',
                    'image_brightness_value': c_img['value'],
                    'image_brightness_interpretation': c_img['brightness'],
                    'image_rgb_avg_r': c_avg_img['rgb_avg_r'],
                    'image_rgb_avg_g': c_avg_img['rgb_avg_g'],
                    'image_rgb_avg_b': c_avg_img['rgb_avg_b'],
                    'image_hsv_avg_h': c_avg_img['hsv_avg_h'],
                    'image_hsv_avg_s': c_avg_img['hsv_avg_s'],
                    'image_hsv_avg_v': c_avg_img['hsv_avg_v'],
                    'object_brightness_value': c_obj['value'],
                    'object_brightness_interpretation': c_obj['brightness'],
                    'object_rgb_avg_r': c_avg_obj['rgb_avg_r'],
                    'object_rgb_avg_g': c_avg_obj['rgb_avg_g'],
                    'object_rgb_avg_b': c_avg_obj['rgb_avg_b'],
                    'object_hsv_avg_h': c_avg_obj['hsv_avg_h'],
                    'object_hsv_avg_s': c_avg_obj['hsv_avg_s'],
                    'object_hsv_avg_v': c_avg_obj['hsv_avg_v'],
                    'detected': det,
                    'confidence': conf,
                    'object_size': bb.get_area(),
                    'object_class': cls
                   }
                  )
  printProgressBar(ix + 1, n_items, prefix = 'Progress:', suffix = 'Complete', length = 50)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:236: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:314: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(


CPU times: user 51min 42s, sys: 43.5 s, total: 52min 26s
Wall time: 43min 5s


In [24]:
df = pd.DataFrame.from_dict(metadata)
df.to_excel(join(OUTPUT_REPORTS, output_testing_report))
df.tail(7)

,image_id,image_version,image_brightness_value,image_brightness_interpretation,image_rgb_avg_r,image_rgb_avg_g,image_rgb_avg_b,image_hsv_avg_h,image_hsv_avg_s,image_hsv_avg_v,object_brightness_value,object_brightness_interpretation,object_rgb_avg_r,object_rgb_avg_g,object_rgb_avg_b,object_hsv_avg_h,object_hsv_avg_s,object_hsv_avg_v,detected,confidence,object_size,object_class
33698,3044,bright,134.776908,light,0.552707,0.519490,0.512211,0.254606,0.208871,0.568104,219.471943,light,0.900724,0.844611,0.839133,0.215730,0.116299,0.918348,False,0.0,10486.0,Bus
33699,3044,contrast,95.425101,dark,0.405319,0.362328,0.354209,0.183127,0.219872,0.419370,198.343621,light,0.790550,0.769863,0.786081,0.294754,0.135345,0.818334,False,0.0,2240.0,Bus
33700,3044,contrast,95.425101,dark,0.405319,0.362328,0.354209,0.183127,0.219872,0.419370,175.065043,light,0.755075,0.657424,0.657277,0.318293,0.271466,0.785582,False,0.0,10486.0,Bus
33701,3044,sharp,82.461668,dark,0.343913,0.316154,0.307026,0.338734,0.255500,0.356855,151.929901,light,0.607832,0.591539,0.586746,0.379261,0.131465,0.630860,False,0.0,2240.0,Bus
33702,3044,sharp,82.461668,dark,0.343913,0.316154,0.307026,0.338734,0.255500,0.356855,137.296597,light,0.574613,0.523310,0.521784,0.416595,0.209666,0.603817,False,0.0,10486.0,Bus
33703,3044,dark,32.575656,dark,0.136002,0.124840,0.121187,0.313693,0.260449,0.140653,60.404664,dark,0.241760,0.235164,0.233139,0.361929,0.113127,0.249445,False,0.0,2240.0,Bus
33704,3044,dark,32.575656,dark,0.136002,0.124840,0.121187,0.313693,0.260449,0.140653,54.514213,dark,0.228268,0.207731,0.207138,0.402688,0.190001,0.238191,False,0.0,10486.0,Bus
